# Greedy Feature Selection
Author: Amir H. Sadeghi
Last Update: 04/10/2023

## Import Libraries

In [6]:
import pandas as pd
import numpy as np
from scipy.stats import bernoulli
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import entropy


## Define Functions

## Read Data

In [5]:
df = pd.read_csv("GenData_LinReg.csv")
m = df.shape[0]
numberOfFeatures = df.shape[1] - 1
df.head()

,x1,x2,x3,corr1,corr2,noise1,noise2,noise3,noise4,noise5,noise6,noise7,noise8,noise9,noise10,y
0,1.439524,3.579187,9.596431,-0.020631,-101.159764,4.338788,2.464870,7.115856,2.837020,2.449958,1.731931,3.032334,4.524291,3.909282,3.213687,28.539186
1,1.769823,5.513767,6.937239,0.683886,-212.185187,3.583505,4.068732,5.230756,2.795022,1.317801,0.865130,2.939818,1.975339,4.422803,2.184427,26.731980
2,3.558708,4.506616,2.204565,-1.620440,-259.446800,4.695768,2.966538,4.796146,0.318664,4.562151,1.712156,2.310784,3.518887,2.675007,3.619724,24.832210
3,2.070508,4.304915,4.629582,-0.327478,-162.221712,4.146469,3.032776,2.446522,2.685872,4.596986,-0.258177,3.006083,3.024128,4.650399,4.298468,21.811595
4,2.129288,3.096763,1.756980,0.822618,-109.831568,4.598500,3.658609,3.389158,4.046742,0.546048,-0.638760,1.507549,2.028737,4.151604,4.274389,16.382843


In [12]:
# Calculate the entropy of each column
entropies = df.apply(lambda x: entropy(x.value_counts(normalize=True)), axis=0)
entropies

x1         4.60517
x2         4.60517
x3         4.60517
corr1      4.60517
corr2      4.60517
noise1     4.60517
noise2     4.60517
noise3     4.60517
noise4     4.60517
noise5     4.60517
noise6     4.60517
noise7     4.60517
noise8     4.60517
noise9     4.60517
noise10    4.60517
y          4.60517
dtype: float64

In [13]:
def solutionGenerator(n,probVector):
    solutionList = []
    for i in range(0,n):
        solution = []
        for j in range(0,len(probVector)):
            solution.append(bernoulli(probVector[j]).rvs())
        #print(solution)
        solutionList.append(solution)
    return solutionList

In [14]:
def modelFit(solutionList):
    df = pd.read_csv("GenData_LinReg.csv")
    mseList = []
    for sol in solutionList:
        X_train, X_test, y_train, y_test = train_test_split(df[['x1', 'x2', 'x3', 'corr1', 'corr2', 'noise1', 'noise2', 'noise3',
       'noise4', 'noise5', 'noise6', 'noise7', 'noise8', 'noise9', 'noise10']], df['y'], test_size=0.3, random_state=42)

        # Convert binary list to list of column indices
        keep_cols = [i for i, x in enumerate(sol) if x]

        # Use the `iloc` indexing function to select columns based on the list of column indices
        X_train = X_train.iloc[:, keep_cols]
        X_test = X_test.iloc[:, keep_cols]

        # Create linear regression object
        model = LinearRegression()

        # Fit the model using the training data
        model.fit(X_train, y_train)

        # Make predictions on the test data
        y_pred = model.predict(X_test)

        # Calculate the mean squared error and R-squared on the test data
        mse = mean_squared_error(y_test, y_pred)
        mseList.append(mse)
    performance = sum(mseList) / len(mseList)
    return performance

# Example

In [52]:
p = [0.5] * numberOfFeatures
p[1] = 1
p[0] = 1
#p[4] = 1
#print("P: ", p)
solList = solutionGenerator(10, p)

mse = modelFit(solList)
print("avg mse: ", mse/(10**7))

avg mse:  1.8487649419674632


In [53]:
solList

[[1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0],
 [1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0],
 [1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0],
 [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0],
 [1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0],
 [1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1],
 [1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1],
 [1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0],
 [1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1],
 [1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0]]

In [56]:
sum_list = [sum(col) for col in zip(*solList)]
sum_list

[10, 10, 5, 4, 4, 6, 4, 6, 3, 7, 6, 4, 5, 5, 3]

In [61]:
min_value = min(sum_list)
max_value = max(sum_list)

portion_list = [(x - min_value) / (max_value - min_value) for x in sum_list]
new_p=[0.5]*len(portion_list)
for n in range(0,len(portion_list)):
    new_p[n] = min(1,p[n]*(1+portion_list[n]))
new_p

[1,
 1,
 0.6428571428571428,
 0.5714285714285714,
 0.5714285714285714,
 0.7142857142857143,
 0.5714285714285714,
 0.7142857142857143,
 0.5,
 0.7857142857142857,
 0.7142857142857143,
 0.5714285714285714,
 0.6428571428571428,
 0.6428571428571428,
 0.5]

In [41]:
p = [0.5] * numberOfFeatures
p[0] = 0
p[1] = 1
print("P: ", p)
solList = solutionGenerator(10, p)

mse = modelFit(solList)
print("avg mse: ", mse/(10**7))

P:  [0, 1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
avg mse:  3.30803642059358


In [42]:
p = [0.5] * numberOfFeatures
p[0] = 0
p[1] = 0
print("P: ", p)
solList = solutionGenerator(10, p)

mse = modelFit(solList)
print("avg mse: ", mse/(10**7))

P:  [0, 0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
avg mse:  6.174035861987858


In [43]:
solList

[[0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1],
 [0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0],
 [0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0],
 [0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1],
 [0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0],
 [0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0],
 [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1],
 [0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1],
 [0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1]]

# Testing Area!

In [40]:
#given branch x1=0
p = [0.5] * numberOfFeatures
def updateP(p):
    k_max = 1
    for k in range(0,k_max):
        for i in range(0,numberOfFeatures):
            for j in range(0,numberOfFeatures):
                print("===================")
                p_i_out = [1]*numberOfFeatures
                p_i_out[i] =
                i_out = solutionGenerator(10, p_i_out)

                p_j_out = [1]*numberOfFeatures
                p_j_out[j] = 0
                j_out = solutionGenerator(10, p_j_out)


                performance_i_out = modelFit(i_out)
                performance_j_out = modelFit(j_out)
                if performance_i_out >= performance_j_out:
                    p[i] *= 1.1
                    p[j] /= 1.1
                print("i=",i,"j=",j)
                print("--")
                print(p)
        print("Finished!: ",p)
    return p

# print
updateP(p)

i= 0 j= 0
--
[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
i= 0 j= 1
--
[0.55, 0.45454545454545453, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
i= 0 j= 2
--
[0.55, 0.45454545454545453, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
i= 0 j= 3
--
[0.6050000000000001, 0.45454545454545453, 0.5, 0.45454545454545453, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
i= 0 j= 4
--
[0.6655000000000002, 0.45454545454545453, 0.5, 0.45454545454545453, 0.45454545454545453, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
i= 0 j= 5
--
[0.7320500000000003, 0.45454545454545453, 0.5, 0.45454545454545453, 0.45454545454545453, 0.45454545454545453, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
i= 0 j= 6
--
[0.8052550000000004, 0.45454545454545453, 0.5, 0.45454545454545453, 0.45454545454545453, 0.45454545454545453, 0.45454545454545453, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
i= 0 j= 7
--
[0.8857805000000005, 0.45454545454545453, 0.5,

[1.5692141883605017,
 0.23325369010486655,
 1.8987491679162074,
 0.13166562715303984,
 0.6050000000000001,
 0.7320500000000003,
 0.5,
 0.8857805000000005,
 1.0717944050000008,
 1.2968712300500012,
 0.28223696502688855,
 0.4132231404958676,
 0.15931540885517823,
 0.34150672768253515,
 0.19277164471476568]

In [16]:
solutionGenerator(5,[0.9,0.0,0.2])

[[1, 0, 1], [1, 0, 1], [1, 0, 1], [1, 0, 0], [1, 0, 0]]

In [ ]:
solArea1
solArea2
solArea3

PerformArea1
PerformArea2
PerformArea3

In [21]:
probVector1 = [0.5] * numberOfFeatures
probVector2 = [0.5] * numberOfFeatures
probVector3 = [0.5] * numberOfFeatures

probVector1[0] = 1
probVector2[0] = 0

if performNotInclude > performInclude:
    probVector1[1] = 1
    probVector2 = probVector1
    probVector2[1] = 0
    probVector3[0] = 1

    if perform3> perform1 or perform3>perform2:
        need back track




for i in includeFeature:
    p[i] = 0


In [130]:
#main code
p = [0.5] * numberOfFeatures

includeFeature = []
notIncludeFeature = []
SurroundingFeature = []

treeLevel = 1
# BUG: The a1 is changing
pInclude = p.copy()
pInclude[treeLevel] = 1
solutionInclude = solutionGenerator(10, pInclude)

pNotInclude = p.copy()
pNotInclude[treeLevel] = 0
solutionNotInclude = solutionGenerator(10, pNotInclude)

# BUG: THE PARTITIONS ---> INCLUDE/EXCLUDE
pSurrounding = p.copy()
pSurrounding[treeLevel-1] = 1
solutionSurrounding = solutionGenerator(10, pSurrounding)

mseInclude = modelFit(solutionInclude)

mseNotInclude = modelFit(solutionNotInclude)

if mseInclude < mseNotInclude:
    p[treeLevel] = min(p[treeLevel]*1.01,1)
else:
    p[treeLevel] = min(p[treeLevel]/1.01,1)

#if (mseInclude+mseNotInclude)/2:
#    print("Need Backtrack")

In [116]:
mseInclude

93640551.65274242

In [117]:
mseNotInclude

259781678.87434483

In [118]:
p

[0.5, 0.505, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]

In [131]:
solutionInclude

[[1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0],
 [0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0],
 [1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1],
 [1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0],
 [1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1],
 [0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1],
 [1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1],
 [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1],
 [0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0]]

In [ ]:
# Meeting


In [26]:

df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
})

# Create an example binary matrix
keep_matrix = np.array([[0, 1, 0],
                        [1, 0, 1],
                        [0, 1, 0]])

mask = keep_matrix.astype(bool)

# Filter dataframe using boolean mask
filtered_df = df.loc[:, mask]

print(filtered_df)

   A  B  B  C
0  1  4  4  7
1  2  5  5  8
2  3  6  6  9


In [27]:
filtered_df

,A,B,B,C
0,1,4,4,7
1,2,5,5,8
2,3,6,6,9


In [28]:
df

,A,B,C
0,1,4,7
1,2,5,8
2,3,6,9


In [33]:
import numpy as np

# assume that X is your dataset with multiple columns
X = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# assume that your list is [0, 1, 0], which means you want to keep the second column
columns_to_keep = np.array([1, 1, 0])

# use boolean indexing to select the desired columns
X_selected = X[:, columns_to_keep == 1]

# X_selected will contain only the second column, which is [2, 5, 8]


In [34]:
X_selected

array([[1, 2],
       [4, 5],
       [7, 8]])

In [40]:
df.columns

Index(['x1', 'x2', 'x3', 'corr1', 'corr2', 'noise1', 'noise2', 'noise3',
       'noise4', 'noise5', 'noise6', 'noise7', 'noise8', 'noise9', 'noise10',
       'y'],
      dtype='object')

In [41]:




# split the data into training and testing sets, with 70% for training and 30% for testing
X_train, X_test, y_train, y_test = train_test_split(df[['x1', 'x2', 'x3', 'corr1', 'corr2', 'noise1', 'noise2', 'noise3',
       'noise4', 'noise5', 'noise6', 'noise7', 'noise8', 'noise9', 'noise10']], df['y'], test_size=0.3, random_state=42)

# print the shapes of the training and testing sets
print('Training set:', X_train.shape, y_train.shape)
print('Testing set:', X_test.shape, y_test.shape)


Training set: (70, 15) (70,)
Testing set: (30, 15) (30,)


In [42]:
X_train

,x1,x2,x3,corr1,corr2,noise1,noise2,noise3,noise4,noise5,noise6,noise7,noise8,noise9,noise10
11,2.359814,6.215929,3.731062,-0.386721,-295.298466,4.320048,3.154055,4.032924,2.439931,3.514007,3.213073,3.157563,4.597132,4.884591,1.414709
47,1.533345,6.375834,-1.079522,6.285834,-250.146623,3.544706,3.288227,7.275300,3.213463,2.349465,-1.202768,1.632797,2.596381,5.213565,4.667392
85,2.331782,4.605648,3.402116,1.657297,-193.316357,6.182717,2.198837,3.838024,1.833315,2.782009,2.195316,2.821002,4.326724,4.058693,3.112965
28,0.861863,3.076287,5.653951,0.401978,-63.135181,3.641696,3.362388,5.238319,3.826738,2.031678,-0.499144,2.315753,1.097209,3.390300,2.995555
93,1.372094,3.209273,1.267596,0.325361,-85.183286,3.381491,2.913268,3.940326,1.862278,2.565382,2.753441,1.590209,3.269253,4.471282,2.944892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,2.379639,7.105423,1.436648,-0.322023,-358.951480,5.201092,3.238713,6.085495,1.932003,1.693633,2.984539,3.863648,4.025494,3.554055,2.191174
71,-0.309169,5.130586,5.887584,-0.181906,-91.586180,3.667775,2.983465,6.200864,3.679109,1.034250,1.177880,2.253661,2.441598,4.506468,1.878988
14,1.444159,6.038814,0.022479,0.895179,-224.110954,3.580249,3.741899,5.696737,3.682397,3.163332,2.491373,2.100411,2.358331,3.250036,3.492047
92,2.238732,5.189167,6.435789,-1.344790,-221.553718,5.416105,2.533642,5.197973,2.793221,2.680752,1.652237,2.416374,1.549938,2.942116,2.431049


In [ ]:
[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [45]:
import numpy as np

# assume that X is your dataset with multiple columns
X = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# assume that your list is [0, 1, 0], which means you want to keep the second column
columns_to_keep = np.array([0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

# use boolean indexing to select the desired columns
X_selected = X_train[:, columns_to_keep == 1]

InvalidIndexError: (slice(None, None, None), array([False, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True]))

In [46]:
import pandas as pd

# Create an example dataframe
df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
})

# Define the binary list of 0s and 1s to keep columns
binary_list = [0, 1, 1] # Keep columns B and C, which correspond to 1s in the binary list

# Convert binary list to list of column names
keep_cols = df.columns[binary_list].tolist()

# Use the `loc` indexing function to select columns based on the list of column names
filtered_df = df.loc[:, keep_cols]

print(filtered_df)


   A  B  B
0  1  4  4
1  2  5  5
2  3  6  6


In [47]:
import pandas as pd

# Create an example dataframe
df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
})

# Define the binary list of 0s and 1s to keep columns
binary_list = [0, 1, 1] # Keep columns B and C, which correspond to 1s in the binary list

# Convert binary list to list of column indices
keep_cols = [i for i, x in enumerate(binary_list) if x]

# Use the `iloc` indexing function to select columns based on the list of column indices
filtered_df = df.iloc[:, keep_cols]

print(filtered_df)


   B  C
0  4  7
1  5  8
2  6  9


In [53]:


# X_train and y_train are your training data
# X_test and y_test are your test data

# Create linear regression object
model = LinearRegression()

# Fit the model using the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate the mean squared error and R-squared on the test data
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the performance metrics
print("Mean squared error (MSE): {:.2f}".format(mse))
print("R-squared (R2): {:.2f}".format(r2))


Mean squared error (MSE): 1128.13
R-squared (R2): -15.79


In [70]:
def modelFit(solutionList):
    df = pd.read_csv("GenData_LinReg.csv")
    performance = 0
    for sol in solutionList:
        X_train, X_test, y_train, y_test = train_test_split(df[['x1', 'x2', 'x3', 'corr1', 'corr2', 'noise1', 'noise2', 'noise3',
       'noise4', 'noise5', 'noise6', 'noise7', 'noise8', 'noise9', 'noise10']], df['y'], test_size=0.3, random_state=42)

        # Convert binary list to list of column indices
        keep_cols = [i for i, x in enumerate(sol) if x]

        # Use the `iloc` indexing function to select columns based on the list of column indices
        X_train = X_train.iloc[:, keep_cols]
        X_test = X_test.iloc[:, keep_cols]

        # Create linear regression object
        model = LinearRegression()

        # Fit the model using the training data
        model.fit(X_train, y_train)

        # Make predictions on the test data
        y_pred = model.predict(X_test)

        # Calculate the mean squared error and R-squared on the test data
        mse = mean_squared_error(y_test, y_pred)
        performance += mse

    return performance





In [71]:
sol_list = solutionGenerator(5,[0.5]*15)
modelFit(df,sol_list)

213743887.42846793

In [67]:
for i in sol_list:
    print("hi",i)

hi [0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1]
hi [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
hi [1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
hi [0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0]
hi [0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1]


,A,B,C
0,1,4,7
1,2,5,8
2,3,6,9


In [ ]:
# Create an example dataframe
df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
})

# Define the binary list of 0s and 1s to keep columns
sol = [0, 1, 1] # Keep columns B and C, which correspond to 1s in the binary list

# Convert binary list to list of column indices
keep_cols = [i for i, x in enumerate(sol) if x]

# Use the `iloc` indexing function to select columns based on the list of column indices
filtered_df = df.iloc[:, keep_cols]